# Introduction

This notebook gives a brief intro to using holoviz-based plotting, incl our own tools that work with it.

To make sure interactivity works correctly, run this notebook in an environment that has all the right stuff installed (i.e., your labcore environment)

## Some basic preliminaries

We assume that `raw` data is in DataDict formats.
For processing and plotting we convert to either `pandas.DataFrame` (for ungridded data) or `xarray.Dataset` (for gridded data).
This approach should be the general way we handle data in the lab.

# Brief primer on vanilla holoviz

We have some tools on top of it for general easy data exploration. 

But for data analysis it's probably very wise for everyone to learn how to use holoviz directly. 
It allows making custom plots extremely easily. 

Docs are here: https://holoviz.org; see in particular ``holoviews`` and `hvplot`.

Here is a very simple example -- we use synthetic Rabi/Chevron data for that.
But much more is possible -- check out the examples in the documentation for how to do different things.

In [1]:
# first, make a dataset in datadict
import numpy as np

from labcore.testing.dispersive_qubit_readout_data import chevron_dataset

raw_data = chevron_dataset(
    Omega_0=1e6,
    Delta_vals=np.linspace(-1e6, 1e6, 11),
    t_vals=np.linspace(0, 3e-6, 25),
    n=500,  # number of shots
)

raw_data

signal: (275, 500)
  ⌙ repetition: (275, 500)
  ⌙ detuning (Hz): (275,)
  ⌙ time (s): (275,)

In [2]:
# next -- convert into xarray. since we know its a nice regular grid, that's the easiest choice here.
from labcore.data.datadict import datadict_to_meshgrid, dd2xr
from labcore.analysis import split_complex

# with the holoviz (or all pydata) tools, complex data isn't always the best, so we split it immediately.
xrdata = split_complex(
    dd2xr(datadict_to_meshgrid(raw_data))
)
xrdata

KeyboardInterrupt: 

In [ ]:
# now we can directly use hvplot on this
import hvplot.xarray  # this import patches xarray objects so we can use hvplot on them.

from bokeh.models import CrosshairTool

# plot: average over repetition, then plot the real part as 2d map.
xrdata.signal_Re.mean('repetition').hvplot.quadmesh(
    x='detuning',
    y='time',
)

In [ ]:
# it's also pretty easy to make linecuts with sliders
_data = xrdata.mean('repetition')

# here: overlay line and scatter plot
# can change x to get a different cut
_data.hvplot.line(x='time') * _data.hvplot.scatter(x='time')

In [ ]:
# looking at readout histograms -- use simply all points in the set
# hexbin is a very simple way to have a quick look at this. 
# doesn't offer too much control, but is 
xrdata.hvplot(
    kind='hexbin',
    aspect=1,
    groupby=[],
    x='signal_Re',
    y='signal_Im',
)

In [ ]:
# if we want to select based on some time/detuning values...
xrdata.hvplot(
    kind='hexbin',
    aspect=1,
    groupby=['time', 'detuning'],
    x='signal_Re',
    y='signal_Im',
    xlim=(-3,3),
    ylim=(-3,3)
)

# Using our own data exploration tools

## The tldr;

A very simple tool for data exploration exists -- it works almost like plottr, but inside jupyter notebooks.

In [ ]:
# a very simple app for exploring typical data
# it has a few issues still -- sometimes it requires going back to setting x and y to None to reset sliders...


from labcore.analysis.plotting.holo import plot_data

plot_data(xrdata.mean('repetition'))

# Prototype

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

from matplotlib import pyplot as plt

import param
from param import Parameter, Parameterized
import panel as pn

import holoviews as hv
hv.extension('bokeh')

import hvplot.xarray
import hvplot.pandas

In [ ]:
from labcore.data.datadict import DataDict, datadict_to_meshgrid, str2dd, dd2df, dd2xr
from labcore.testing.dispersive_qubit_readout_data import rabi, chevron_dataset
from labcore.analysis import split_complex

raw_data = chevron_dataset(
    Omega_0=1e6,
    Delta_vals=np.linspace(-1e6, 1e6, 11),
    t_vals=np.linspace(0, 3e-6, 25),
    n=500,  # number of shots
)

xrdata = split_complex(
    dd2xr(datadict_to_meshgrid(raw_data))
)
xrdata

## inspect with own app

In [ ]:
from labcore.analysis.plotting.holo import Node, LoaderNodeBase, LoaderNodePath, ValuePlot, ComplexHist
from labcore.data.datadict_storage import datadict_from_hdf5

class RabiSimulation(LoaderNodeBase):
    def load_data(self) -> DataDict:
        Omega_0 = 1
        Delta_vals = np.linspace(-2, 2, 41)
        t_vals = np.linspace(0, 3, 31)
        nreps = 50
        dd = chevron_dataset(Omega_0, Delta_vals, t_vals, nreps).expand()
        return dd       
s = RabiSimulation(name='sim')
#s = LoaderNodePath(name = 'sim', path = 'Specify Path Here!')
app = pn.Column(
    s,
    s.plot,
)
app

In [ ]:
p = ComplexHist(name='testplot', data_in=s.data_out)
p

In [ ]:
from labcore.measurement.sweep import Sweep, sweep_parameter
from labcore.measurement.record import record_as
import os

from labcore.analysis.plotting.holo import LoaderNodeSweep


def square_fn(x):
    return x**2 - 6*x + 5

my_sweep = sweep_parameter('x', np.linspace(-10, 10), record_as(square_fn, 'y'))

s = LoaderNodeSweep(input_sweep=my_sweep, name = 'experiment', sweep_func = None)
app = pn.Column(
    s,
    s.plot,
)
app